# Crop Health Monitoring

In [1]:
import os
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import numpy as np
import tensorflow as tf
import pandas as pd
pd.options.mode.chained_assignment = None
import plotly.express as px
import plotly.graph_objects as go
import warnings
import random
import datetime
import requests
warnings.filterwarnings('ignore')
from tensorflow.python.util import deprecation
from email_validator import validate_email, EmailNotValidError
deprecation._PRINT_DEPRECATION_WARNINGS = False

np.random.seed(1)

In [2]:
r1 = {"January":{"NDVI":{"min":0.61,"max":0.75},"EVI":{"min":1.233,"max":1.550}},
      "February":{"NDVI":{"min":0.71,"max":0.85},"EVI":{"min":1.460,"max":1.777}},
      "March":{"NDVI":{"min":0.725,"max":0.835},"EVI":{"min":1.494,"max":1.743}},
      "April":{"NDVI":{"min":0.295,"max":0.415},"EVI":{"min":0.519,"max":0.791}},
      "May":{"NDVI":{"min":0.101,"max":0.151},"EVI":{"min":0.079,"max":0.192}},
      "June":{"NDVI":{"min":0.155,"max":0.215},"EVI":{"min":0.201,"max":0.337}},
      "July":{"NDVI":{"min":0.3,"max":0.325},"EVI":{"min":0.530,"max":0.587}},
      "August":{"NDVI":{"min":0.595,"max":0.715},"EVI":{"min":1.199,"max":1.471}},
      "September":{"NDVI":{"min":0.565,"max":0.72},"EVI":{"min":1.131,"max":1.482}},
      "October":{"NDVI":{"min":0.41,"max":0.55},"EVI":{"min":0.780,"max":1.097}},
      "November":{"NDVI":{"min":0.101,"max":0.215},"EVI":{"min":0.079,"max":0.337}},
      "December":{"NDVI":{"min":0.315,"max":0.487},"EVI":{"min":0.564,"max":0.954}}}

r2 = {"January":{"NDVI":{"min":0.585,"max":0.708},"EVI":{"min":1.176,"max":1.455}},
      "February":{"NDVI":{"min":0.685,"max":0.831},"EVI":{"min":1.403,"max":1.734}},
      "March":{"NDVI":{"min":0.699,"max":0.825},"EVI":{"min":1.435,"max":1.720}},
      "April":{"NDVI":{"min":0.285,"max":0.405},"EVI":{"min":0.496,"max":1.768}},
      "May":{"NDVI":{"min":0.094,"max":0.145},"EVI":{"min":0.063,"max":0.179}},
      "June":{"NDVI":{"min":0.145,"max":0.21},"EVI":{"min":0.0179,"max":0.326}},
      "July":{"NDVI":{"min":0.285,"max":0.362},"EVI":{"min":0.496,"max":0.671}},
      "August":{"NDVI":{"min":0.585,"max":0.705},"EVI":{"min":1.176,"max":1.448}},
      "September":{"NDVI":{"min":0.555,"max":0.715},"EVI":{"min":1.108,"max":1.471}},
      "October":{"NDVI":{"min":0.401,"max":0.503},"EVI":{"min":0.759,"max":0.990}},
      "November":{"NDVI":{"min":0.121,"max":0.259},"EVI":{"min":0.124,"max":0.437}},
      "December":{"NDVI":{"min":0.32,"max":0.475},"EVI":{"min":0.575,"max":0.927}}}

r3 = {"January":{"NDVI":{"min":0.618,"max":0.746},"EVI":{"min":1.251,"max":1.541}},
      "February":{"NDVI":{"min":0.718,"max":0.845},"EVI":{"min":1.478,"max":1.766}},
      "March":{"NDVI":{"min":0.715,"max":0.827},"EVI":{"min":1.471,"max":1.725}},
      "April":{"NDVI":{"min":0.299,"max":0.42},"EVI":{"min":0.528,"max":0.802}},
      "May":{"NDVI":{"min":0.102,"max":0.155},"EVI":{"min":0.081,"max":0.201}},
      "June":{"NDVI":{"min":0.149,"max":0.22},"EVI":{"min":0.188,"max":0.349}},
      "July":{"NDVI":{"min":0.31,"max":0.321},"EVI":{"min":0.553,"max":0.578}},
      "August":{"NDVI":{"min":0.601,"max":0.719},"EVI":{"min":1.213,"max":1.480}},
      "September":{"NDVI":{"min":0.559,"max":0.729},"EVI":{"min":1.117,"max":1.503}},
      "October":{"NDVI":{"min":0.415,"max":0.562},"EVI":{"min":0.791,"max":1.124}},
      "November":{"NDVI":{"min":0.111,"max":0.249},"EVI":{"min":0.102,"max":0.415}},
      "December":{"NDVI":{"min":0.331,"max":0.495},"EVI":{"min":0.600,"max":0.972}}}

In [3]:
model = tf.keras.models.load_model('anamoly.h5')

In [4]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.remove_control(Map.inspector_control)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

**Instructions:**<br />
**To check already marked Area of Interests, follow these instructions:**
1. Choose an area in 'Area of Interest' and 'Sensitivity lower threshold'.
2. Choose 'Start Year' and 'Number of years'.
3. Click 'Submit'.

**To mark & check your own area of interests, follow these instructions:**
1. Mark your Area of Interest in the Map.
2. Choose 'My Region' in 'Area of Interest', and 'Sensitivity lower threshold'.
3. Choose 'Start Year' and 'Number of years'.
4. Click 'Submit'.

**System will analyze the Satellite images and display results as below:**
1. Display Health graph(s) for the area chosen along with identified health anomalies.
2. Health anomalies are categorized into 'mild' (variation upto Sensitivity lower threshold), 'medium' (variation upto 10% of Sensitivity lower threshold) & 'severe' (variation beyond 10% of Sensitivity lower threshold).
3. Hover mouse or tap on graph to show more controls to analyze the graph better.
4. Months which have anomaly will be highlighted on the Map as well as separate layers for closer examination.
5. System predicted values will be used for months where Satellite images are unavailable or unusable so that the health of farm can be depicted for the overall chosen duration.
6. If more than 40% of the requested duration (in months) have unusable or no Satellite images, system will not 'assess' health of that farm to show abnormalities, but will just show the health status as is.

In [5]:
def regionReduce(img, props):
  date = img[props[0]]
  
  
  stat = img[props[1]].reduceRegion(
    reducer = reReArgs['reducer'],
    geometry = reReArgs['geometry'],
    scale = reReArgs['scale'])

  return(ee.Feature(None, stat).copyProperties(img[props[1]], img[props[1]].propertyNames()).set({'Date': date}))

In [6]:
def getReReList(col, props):
    d = []
    for i in col:
        r = regionReduce(i,props)
        d.append(r)
    da = ee.FeatureCollection(d)
    dict = {}
    dict =  da.filter(ee.Filter.notNull(props)).reduceColumns(reducer = ee.Reducer.toList().repeat(len(props)),selectors = props)

    return (ee.List(dict.get('list')).getInfo())

In [7]:
def create_seq(X,y,time_steps=1):
    Xs,ys=[],[]
    for i in range(len(X)-time_steps):
        Xs.append(X.iloc[i:(i+time_steps)].values)
        ys.append(y.iloc[i+time_steps])
    return np.array(Xs), np.array(ys)

In [42]:
def chkim(aug):
    ln = len(aug)
    x = aug.count(True)
    pr = (0.45 * ln)
    if (x == ln):
        return("100p")
    elif (x >= int(round(pr))):
        return("40p")
    else:
        return("l40p")

In [9]:
def fill_values(df):
    x = random.choice([r1,r2,r3])
    aug = []
    for i in range(len(df.index)):
      if pd.isna(df.at[i,'EVI']):
        date = df.at[i,'date']
        month = date.strftime("%B")
        df.at[i,'EVI'] = random.uniform(x[month]['EVI']['min'],x[month]['EVI']['max'])
      if pd.isna(df.at[i,'NDVI']):
        date = df.at[i,'date']
        month = date.strftime("%B")
        df.at[i,'NDVI'] = random.uniform(x[month]['NDVI']['min'],x[month]['NDVI']['max'])
        aug.append(True)
      else:
        aug.append(False)
    df['aug'] = aug
    st = chkim(aug)
    return df,st

In [10]:
def create_df(NDVI,EVI):
    NDVI = pd.DataFrame(NDVI)
    NDVI = NDVI.transpose()
    NDVI.columns = ['Date', 'NDVI']
    NDVI['Date'] = pd.to_datetime(NDVI['Date'])
    NDVI = NDVI.set_index('Date')
    EVI = pd.DataFrame(EVI)
    EVI = EVI.transpose()
    EVI.columns = ['Date', 'EVI']
    EVI['Date'] = pd.to_datetime(EVI['Date'])
    EVI = EVI.set_index('Date')
    df = pd.concat([NDVI,EVI],axis=1)
    to_date = str(Start_year.value+Nmbry.value)+'-12-31'
    t_d = str(Start_year.value+Nmbry.value)+'-12'
        
    td = datetime.datetime.strptime(t_d, "%Y-%m")
    x = datetime.datetime.today()
    x = x.strftime("%Y-%m")
    x = datetime.datetime.strptime(x, "%Y-%m")
        
    if td>x:
        date = x.strftime("%Y-%m")
        to_date = date+"-01"
    dates = pd.date_range(start=(str(Start_year.value-1)+'-05-01'),end=(to_date),freq='MS')
    df = df.reindex(dates)
    df = df.infer_objects()
    df = df.reset_index()
    df = df.rename(columns={"index":"date"})
    df,st = fill_values(df)
    if (st == "40p"):
        df2 = pd.DataFrame(df[8:])
        df3 = df2[df2.aug==False]
        return df3,st
    else:
        return df,st

In [11]:
def predict(df):
    X,y = create_seq(df[['NDVI','EVI']],df.NDVI,8)
    X_pred= model.predict(X)
    mae_loss = np.abs(X_pred[:,0] - y)
    
    ndvi_thresholdn = (thrsld.value/100)
    ndvi_thresholdl = ((thrsld.value/100)+0.1)
    ndvi_thresholdu = ((thrsld.value/100)+0.2)
    
    score_df = pd.DataFrame(df[8:])

    score_df['loss_NDVI'] = mae_loss
    
    score_df['threshold_NDVIn'] = ndvi_thresholdn
    
    score_df['threshold_NDVIl'] = ndvi_thresholdl
    
    score_df['threshold_NDVIu'] = ndvi_thresholdu
    
    score_df['anomaly_NDVIn'] = ((score_df.loss_NDVI > score_df.threshold_NDVIn) & (score_df.loss_NDVI < score_df.threshold_NDVIl) & (score_df.loss_NDVI < score_df.threshold_NDVIu))

    score_df['anomaly_NDVIl'] = ((score_df.loss_NDVI > score_df.threshold_NDVIl) & (score_df.loss_NDVI < score_df.threshold_NDVIu))
    
    score_df['anomaly_NDVIu'] = score_df.loss_NDVI > score_df.threshold_NDVIu
    
    score_df['NDVI'] = df[8:].NDVI
    
    score_df['prediction'] = X_pred[:,0]
    
    anomalies_NDVIn = score_df[score_df.anomaly_NDVIn == True]
    
    anomalies_NDVIl = score_df[score_df.anomaly_NDVIl == True]
    
    anomalies_NDVIu = score_df[score_df.anomaly_NDVIu == True]
    
    anomalies_NDVIa = score_df[score_df.aug == True]
    
    return anomalies_NDVIl,anomalies_NDVIu,anomalies_NDVIn,anomalies_NDVIa,score_df

In [12]:
def aplot(andvil,andviu,andvin,andvia,sdf,farm):
    from_date = str(Start_year.value)+'-01-01'
    to_date = str(Start_year.value+Nmbry.value)+'-12-31'
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=andvin.date, y=andvin.NDVI,mode='markers',name='Mild abnormality',marker = dict(color = 'rgb(253, 229, 52)',size = 15)))
    fig.add_trace(go.Scatter(x=andvil.date, y=andvil.NDVI,mode='markers',name='Medium abnormality',marker = dict(color = 'rgb(235, 153, 52)',size = 15)))
    fig.add_trace(go.Scatter(x=andviu.date, y=andviu.NDVI,mode='markers',name='Severe abnormality',marker = dict(color = 'rgb(235, 52, 52)',size = 15)))
    fig.add_trace(go.Scatter(x=andvia.date, y=andvia.NDVI,mode='markers',name='Predicted data',marker = dict(color = 'rgb(0, 0, 0)',size = 10)))
    fig.add_trace(go.Scatter(x=sdf.date, y=sdf.NDVI,mode='lines',name='Health Graph',line = dict(color="#00fc41",width=4)))
    fig.update_layout(title="Health Graph for "+farm+" between "+from_date+" and "+to_date,showlegend=True,font=dict(family="Courier New, monospace",size=18,color="RebeccaPurple"))
    fig.show()
    return

In [13]:
def bplot(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.date, y=df.NDVI,mode='markers',name='Halth value',marker = dict(color = 'rgb(0, 0, 0)',size = 15)))
    fig.update_layout(title="Health Graph",showlegend=True,font=dict(family="Courier New, monospace",size=18,color="RebeccaPurple"))
    fig.show()
    return

In [14]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))

    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

In [15]:
def EVI(image):
    return image.expression(
        '(2.5*(NIR-R))/(NIR+(2.4*R)+1)', 
        {
        'NIR': image.select('B4'),
        'R': image.select('B3')
        })

In [16]:
def ProcessImg(region,from_date,to_date,clouds_percentage):
    
    NUM_OF_MONTHS = ee.Date(to_date).difference(from_date, 'month')
    
    region_to_clip = ee.FeatureCollection(region)

    NUM_OF_MONTHS_TO_DISPLAY = NUM_OF_MONTHS.round()

    Filtered_Collection = Landsat7_tier1_dataset.filterBounds(region_to_clip)

    filteredRegion_Date_month_to_display = Filtered_Collection.filterDate(from_date, to_date)
    
    GLOBAL_NDVI = []
    GLOBAL_EVI = []
    delta = 0
    j = 0
    for i in range(NUM_OF_MONTHS_TO_DISPLAY.getInfo()):
        from_date_next = ee.Date(from_date).advance(delta, 'month')
        to_date_next = from_date_next.advance(30, 'day')
        filteredRegion_Date_month = Filtered_Collection.filterDate(from_date_next, to_date_next)
        filteredRegion_Date_month_unmastCloud = filteredRegion_Date_month.map(cloudMaskL457)

        numImg = filteredRegion_Date_month_unmastCloud.size().getInfo()

        if(numImg < 1):
            delta+=1
            continue
        
        medianRGB = filteredRegion_Date_month_unmastCloud.median()
        medianNDVI = medianRGB.normalizedDifference(['B4','B3'])
        NDVI_VALUE_DICT = {"Date": from_date_next.format('dd MMM YYYY').getInfo(), "NDVI": ee.Image(medianNDVI).select([0], ['NDVI'])}
        GLOBAL_NDVI.append(NDVI_VALUE_DICT)

        medianEV = filteredRegion_Date_month_unmastCloud.map(EVI)
        medianEVI = medianEV.median()
        EVI_VALUE_DICT = {"Date": from_date_next.format('dd MMM YYYY').getInfo(), "EVI": ee.Image(medianEVI).select([0], ['EVI'])}
        GLOBAL_EVI.append(EVI_VALUE_DICT)
        
        delta+=1
        j+=1
        
    return GLOBAL_NDVI,GLOBAL_EVI

In [17]:
style = {'description_width': 'initial'}

In [18]:
Crops = widgets.Dropdown(
        options=['Sugarcane'],
        value='Sugarcane',
        description='Crop type:',
        disabled=False,
        style = style
        )

In [19]:
Areas = widgets.Dropdown(
        options=['Amloh', 'Batala','Bhogpur','Bhudewal','Buttarsevan','Dhuri','Doaba','Gurdaspur','Majha','Phagwara','My_Region'],
        value='Amloh',
        description='Select Area of Interest:',
        disabled=False,
        style = style
        )

In [20]:
Start_year = widgets.IntSlider(
        layout= widgets.Layout(width='70%'),
        value = 2001,
        min = 2001,
        max = 2019,
        step = 1,
        style = style,
        description = 'Start Year:')

In [21]:
Nmbry = widgets.BoundedIntText(
    value=1,
    min=0,
    max=18,
    step=1,
    description='Number Of years:',
    disabled=False,
    style = style
)

In [22]:
thrsld = widgets.IntSlider(
        layout= widgets.Layout(width='50%'),
        value = 20,
        min = 10,
        max = 30,
        step = 1,
        style = style,
        description = 'Choose Sensitivity lower threshold value')

In [23]:
Chckbx = widgets.Checkbox(
    value=False,
    description='Notify me every week',
    disabled=True,
    indent=False
)

In [24]:
Txtbx = widgets.Text(
    value='',
    placeholder='Enter your mial id',
    description='Mail id:',
    disabled=True
)

In [25]:
Txtbx2 = widgets.Text(
    value= '',
    placeholder='Name of the farm',
    description='Farm Name:',
    disabled=True
)

In [26]:
Hbox = widgets.HBox([Areas,Crops,Txtbx2])
Hbox

In [27]:
thrsld

IntSlider(value=20, description='Choose Sensitivity lower threshold value', layout=Layout(width='50%'), max=30…

In [28]:
Hbox1 = widgets.HBox([Start_year,Nmbry])
Hbox1

In [29]:
Hbox2 = widgets.HBox([Chckbx,Txtbx])
Hbox2

In [30]:
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 'My_Region':
            Chckbx.disabled = False
            Txtbx.disabled = False
            Txtbx2.disabled = False
        else:
            Chckbx.disabled = True
            Txtbx.disabled = True
            Txtbx2.disabled = True

Areas.observe(on_change)

In [31]:
def Imageurl(sdf,geometry,region,farm):
    
    from_date = str(Start_year.value)+'-01-01'
    to_date = str(Start_year.value+Nmbry.value)+'-12-31'
    t_d = str(Start_year.value+Nmbry.value)+'-12'
        
    td = datetime.datetime.strptime(t_d, "%Y-%m")
    x = datetime.datetime.today()
    x = x.strftime("%Y-%m")
    x = datetime.datetime.strptime(x, "%Y-%m")
        
    if td>x:
        date = x.strftime("%Y-%m")
        to_date = date+"-01"
    clouds_percentage = 100
    
    n = sdf.anomaly_NDVIn.values.tolist()
    s = sdf.anomaly_NDVIl.values.tolist()
    t = sdf.anomaly_NDVIu.values.tolist()
    
    NUM_OF_MONTHS = ee.Date(to_date).difference(from_date, 'month')

    
    region_to_clip = ee.FeatureCollection(region)

    NUM_OF_MONTHS_TO_DISPLAY = NUM_OF_MONTHS.round()

    Filtered_Collection = Landsat7_tier1_dataset.filterBounds(region_to_clip)

    filteredRegion_Date_month_to_display = Filtered_Collection.filterDate(from_date, to_date)
    
    delta = 0
    for i in range(NUM_OF_MONTHS_TO_DISPLAY.getInfo()):
        from_date_next = ee.Date(from_date).advance(delta, 'month')
        to_date_next = from_date_next.advance(30, 'day')
    
        if s[delta] == True:
            a = ee.Image().paint(geometry,1)
            Map.addLayer(a, {'max': 3, 'palette': ['red', '#eb8334', 'yellow', 'blue'],'opacity':0.2},str(from_date_next.format('MMM yyyy').getInfo())+' farm'+' '+str(farm), False)
            delta+=1
            continue
            
        elif t[delta] ==True:
            a = ee.Image().paint(geometry,0)
            Map.addLayer(a, {'max': 3, 'palette': ['red', '#eb8334', 'yellow', 'blue'],'opacity':0.2},str(from_date_next.format('MMM yyyy').getInfo())+' farm'+' '+str(farm), False)
            delta+=1
            continue
            
        elif n[delta] ==True:
            a = ee.Image().paint(geometry,2)
            Map.addLayer(a, {'max': 3, 'palette': ['red', '#eb8334', 'yellow', 'blue'],'opacity':0.2},str(from_date_next.format('MMM yyyy').getInfo())+' farm'+' '+str(farm), False)
            delta+=1
            continue
        delta+=1
        
    return

In [32]:
Submit = widgets.Button(
        description = 'Submit',
        button_syle = 'primary',
        tooltip = 'Press Button To Submit Selection')

In [33]:
Center = widgets.Button(
        description = 'Reset to region',
        button_syle = 'primary',
        tooltip = 'Press Button To Zoom To Last Selected Region')

In [34]:
def submit_clck(a):
    with output:
        clear_output()
        f.value = 0
        if Chckbx.value==True:
            try:
                valid = validate_email(Txtbx.value)
                email = valid.email
            except EmailNotValidError as e:
                print("The email you have entered is not valid please enter a valid email to proceed further")
                return
        if Areas.value != 'My_Region':
            Map.clear_layers()
            Map.add_basemap('SATELLITE')
        print('Analysing satellite images...please wait')

        clouds_percentage = 100
        from_date = str(Start_year.value-1)+'-05-01'
        to_date = str(Start_year.value+Nmbry.value)+'-12-31'
        t_d = str(Start_year.value+Nmbry.value)+'-12'
        
        td = datetime.datetime.strptime(t_d, "%Y-%m")
        x = datetime.datetime.today()
        x = x.strftime("%Y-%m")
        x = datetime.datetime.strptime(x, "%Y-%m")
        
        if td>x:
            date = x.strftime("%Y-%m")
            to_date = date+"-01"
        if Areas.value == 'Amloh':

            bigger_region = ee.Geometry.Polygon([[76.25085426860677,30.620745545307166],[76.26274181896078,30.620745545307166],
                                                          [76.26274181896078,30.628500771520102],[76.25085426860677,30.628500771520102],
                                                          [76.25085426860677,30.620745545307166]])

            region1 =  ee.Geometry.Polygon([[76.25164475711391,30.62288189965434],[76.25158038409755,30.62225408229643],[76.2517520454745,30.62225408229643],
                                           [76.25168767245815,30.62159394641798],[76.25206318172023,30.621580097365317],[76.25212755473659,30.62222176805753],
                                           [76.25222947867915,30.622240233338218],[76.25231530936763,30.622775724946102],[76.25165012153194,30.622844969334242],
                                           [76.25164475711391,30.62288189965434]])

            region2 = ee.Geometry.Polygon([[76.26040404174972,30.623332200494634],[76.26025383804489,30.622330465229815],[76.26139645908523,30.622205824632942],
                                          [76.26136427257705,30.62259359485169],[76.26140182350326,30.623221410007506],[76.26040404174972,30.623332200494634]]) 

            region3 = ee.Geometry.Polygon([[76.25230006190195,30.626703541908753],[76.25349096270456,30.626925114784342],[76.2528686902131,30.628116060299334],
                                          [76.2528686902131,30.628116060299334],[76.25230006190195,30.626703541908753]]) 

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")

        elif Areas.value == 'Batala':

            bigger_region = ee.Geometry.Polygon([[75.19686800638378,31.858310350453834],[75.20446402231396,31.858310350453834],[75.20446402231396,31.864105782474493],
                                                [75.19686800638378,31.864105782474493],[75.19686800638378,31.858310350453834]])

            region1 =  ee.Geometry.Polygon([[75.19942146936596,31.860376609557367],[75.19944292703808,31.8597888580974],[75.20105225244701,31.85975696450016],
                                           [75.20104152361095,31.86035382856284],[75.19942146936596,31.860376609557367]])

            region2 = ee.Geometry.Polygon([[75.20223242441357,31.861424529220724],[75.20226461092174,31.860039450263372],[75.20286542574108,31.860044006478255],
                                          [75.20288151899517,31.86144275380513],[75.20223242441357,31.861424529220724]]) 

            region3 = ee.Geometry.Polygon([[75.1975922028178,31.863092063777618],[75.19758147398174,31.86263190015107],[75.19924444357098,31.862654680582665],
                                          [75.199249807989,31.863096619841638],[75.1975922028178,31.863092063777618]]) 

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")
            
        elif Areas.value == 'Bhogpur':

            bigger_region = ee.Geometry.Polygon([[75.65630873018264,31.551446717424277],[75.66776712709427,31.551446717424277],[75.66776712709427,31.559400613789183],
                                                [75.65630873018264,31.559400613789183],[75.65630873018264,31.551446717424277]])

            region1 =  ee.Geometry.Polygon([[75.6586142819324,31.55782580586668],[75.65863037518649,31.55808407193102],[75.65916413478045,31.558061216555917],
                                           [75.65915072373538,31.55779609379571],[75.6598400514522,31.557789237162623],[75.65974349192767,31.5583560504647],
                                           [75.65956110171466,31.558714878073264],[75.65811539105563,31.558730876724287],[75.65810466221957,31.55784409021335],
                                           [75.6586142819324,31.55782580586668]])

            region2 = ee.Geometry.Polygon([[75.65701836756854,31.55645446965383],[75.65727585963397,31.556447612922117],[75.65729463509707,31.555864788884634],
                                          [75.65784717015414,31.555835076189112],[75.65785253457217,31.556964151966728],[75.6569942276874,31.556971008660476],
                                          [75.65701836756854,31.55645446965383]]) 

            region3 = ee.Geometry.Polygon([[75.6649990873909,31.553448968993123],[75.66468795114517,31.552562132274815],[75.66587885194778,31.552278708453436],
                                          [75.66625503032496,31.55219756621795],[75.66631470919516,31.55261927327636],[75.66634287254782,31.553032979141395],
                                          [75.66657086187362,31.5531198346999],[75.66555162244796,31.55332097246151],[75.6654496985054,31.55313354865196],
                                          [75.66508491807937,31.553188404440053],[75.66510101133346,31.553403255966277],[75.6649990873909,31.553448968993123]]) 

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")

        elif Areas.value == 'Bhudewal':

            bigger_region = ee.Geometry.Polygon([[75.99756103317735,30.910111424694012],[76.0058007792711,30.910111424694012],[76.0058007792711,30.920346909739305],
                                                [75.99756103317735,30.920346909739305],[75.99756103317735,30.910111424694012]])

            region1 =  ee.Geometry.Polygon([[75.99903360617417,30.91093096280828],[76.00023523581284,30.91090334784512],[76.00017086279648,30.912698303873647],
                                            [75.99908725035446,30.912744327944445],[75.99903360617417,30.91093096280828]])

            region2 = ee.Geometry.Polygon([[76.00325365630694,30.917702726707873],[76.00324829188891,30.917040012297818],[76.00525458423205,30.917049216696075],
                                          [76.00517948237963,30.917675113699083],[76.00325365630694,30.917702726707873]]) 

            region3 = ee.Geometry.Polygon([[76.00198913716831,30.92041624930465],[76.00202132367649,30.919012617745825],[76.00324441098728,30.919500439573515],
                                          [76.00318003797092,30.92040704523038],[76.00198913716831,30.92041624930465]])

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")

        elif Areas.value == 'Buttarsevan':

            bigger_region = ee.Geometry.Polygon([[75.27843631189968,31.628243996330756],[75.28796351832058,31.628243996330756],[75.28796351832058,31.633834611297736],
                                                [75.27843631189968,31.633834611297736],[75.27843631189968,31.628243996330756]])

            region1 =  ee.Geometry.Polygon([[75.27950873633282,31.62993836606439],[75.27948996086971,31.62922354175095],[75.28017660637752,31.629221257958392],
                                           [75.28019269963161,31.629340015096723],[75.28077205677883,31.629360569201435],[75.28087666293041,31.629913244536265],
                                           [75.27950873633282,31.62993836606439]])

            region2 = ee.Geometry.Polygon([[75.28633407619027,31.631442825807454],[75.28633658552134,31.6308423226609],[75.2871224727627,31.630826336390044],
                                          [75.28711442613566,31.631456650130588],[75.28633407619027,31.631442825807454]]) 

            region3 = ee.Geometry.Polygon([[75.2816575049415,31.633496171481006],[75.281566309835,31.63299375884668],[75.28226368417887,31.632984624046387],
                                          [75.28222076883463,31.633496171481006],[75.2816575049415,31.633496171481006]])

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")
            
        elif Areas.value == 'Doaba':

            bigger_region = ee.Geometry.Polygon([[75.49253283242797,32.13611057302962],[75.49910960893249,32.13611057302962],[75.49910960893249,32.14037135594249],
                                                [75.49253283242797,32.14037135594249],[75.49253283242797,32.13611057302962]])

            region1 =  ee.Geometry.Polygon([[75.49720764733144,32.13728818293632],[75.49779236889668,32.137265470633785],[75.49778700447865,32.137724258048316],
                                           [75.49719691849538,32.13776059754687],[75.49720764733144,32.13728818293632]])

            region2 = ee.Geometry.Polygon([[75.49697161293813,32.138977962381105],[75.49785137749501,32.13899613188242],[75.49798012352773,32.139804671026326],
                                          [75.4972720203478,32.140049956765125],[75.49642444229909,32.139105148814295],[75.49697161293813,32.138977962381105]]) 

            region3 = ee.Geometry.Polygon([[75.49294052819823,32.13735521463155],[75.49349842767333,32.13729162021693],[75.49361108045196,32.138808789161466],
                                          [75.49291370610808,32.138826958696505],[75.49294052819823,32.13735521463155]])

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")
            
        elif Areas.value == 'Dhuri':

            bigger_region = ee.Geometry.Polygon([[75.83758675353037,30.38927782520813],[75.84612690703379,30.38927782520813],[75.84612690703379,30.393590459171506],
                                                [75.83758675353037,30.393590459171506],[75.83758675353037,30.38927782520813]])

            region1 =  ee.Geometry.Polygon([[75.83887154682162,30.391336401451174],[75.83893591983798,30.38977236346087],[75.83956355674746,30.389763108723695],
                                           [75.83945626838687,30.391345656039316],[75.83887154682162,30.391336401451174]])

            region2 = ee.Geometry.Polygon([[75.84374243839267,30.3908690436098],[75.84374243839267,30.389726089766196],[75.84398383720401,30.389721462395514],
                                          [75.84398383720401,30.390110160768074],[75.84457392318728,30.390133297527214],[75.84456855876925,30.3903646648170780],
                                          [75.84482605083468,30.390378546837052],[75.84483141525271,30.391114291072682],[75.84374243839267,30.3908690436098]]) 

            region3 = ee.Geometry.Polygon([[75.83809100882517,30.39321565626314],[75.83817147509562,30.39249843681239],[75.8387079168986,30.39254008195675],
                                          [75.83876692549693,30.392105120684363],[75.83962523238169,30.39233648330441],[75.83956085936533,30.393164756991844],
                                          [75.83809100882517,30.39321565626314]])

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")

        elif Areas.value == 'Gurdaspur':

            bigger_region = ee.Geometry.Polygon([[75.44608868540088,31.111766632428527],[75.4535130399541,31.111766632428527],[75.4535130399541,31.11904135748964],
                                                [75.44608868540088,31.11904135748964],[75.44608868540088,31.111766632428527]])

            region1 =  ee.Geometry.Polygon([[75.4499889412982,31.114658046121267],[75.45002112780638,31.113160825343456],[75.45088479910918,31.113147047128493],
                                            [75.45084188376494,31.113569578144606],[75.45059512053557,31.113578763580595],[75.45057366286345,31.11463508279071],
                                           [75.4499889412982,31.114658046121267]])

            region2 = ee.Geometry.Polygon([[75.44760177527495,31.114814196621662],[75.44764469061919,31.113202159976368],[75.4484225312335,31.11319297450394],
                                          [75.44844398890562,31.114814196621662],[75.44760177527495,31.114814196621662]]) 

            region3 = ee.Geometry.Polygon([[75.45039582389578,31.117819729300397],[75.45035290855154,31.117571733297147],[75.45197296279653,31.11735588439653],
                                          [75.45202660697683,31.117704916416482],[75.45269715923055,31.117654398703554],[75.45269179481252,31.11817794460557],
                                          [75.4517691149114,31.118210092066825],[75.4516779198049,31.117737064037982],[75.45039582389578,31.117819729300397]])

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")

        elif Areas.value == 'Majha':

            bigger_region = ee.Geometry.Polygon([[76.08707814282226,30.28907874266941],[76.09628348416138,30.28907874266941],[76.09628348416138,30.296267545701124],
                                                [76.08707814282226,30.296267545701124],[76.08707814282226,30.28907874266941]])

            region1 =  ee.Geometry.Polygon([[76.09134285515594,30.29233507633397],[76.09133212631988,30.2913253002182],[76.09192757672119,30.291306772118958],
                                           [76.09202413624573,30.292316548425482],[76.09134285515594,30.29233507633397]])

            region2 = ee.Geometry.Polygon([[76.09138748598862,30.293634368278404],[76.09135529948044,30.29299516188055],[76.09203658057022,30.292958106309445],
                                          [76.09211704684067,30.293555625686285],[76.09138748598862,30.293634368278404]]) 

            region3 = ee.Geometry.Polygon([[76.08794555320918,30.293767672575346],[76.08790800228297,30.293146994801422],[76.08832106247127,30.293137730924297],
                                          [76.0883639778155,30.293735249206776],[76.08794555320918,30.293767672575346]])

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")

        elif Areas.value == 'Phagwara':

            bigger_region = ee.Geometry.Polygon([[75.72534922656267,31.229225809997722],[75.73041323718279,31.229225809997722],[75.73041323718279,31.23315229746711],
                                                [75.72534922656267,31.23315229746711],[75.72534922656267,31.229225809997722]])

            region1 =  ee.Geometry.Polygon([[75.72607342299669,31.232725712686943],[75.72604660090654,31.231785384595522],[75.72664741572588,31.231785384595522],
                                           [75.72670105990618,31.232482604370027],[75.72651866969316,31.232496365234837],[75.7265240341112,31.232711951855514],
                                           [75.72607342299669,31.232725712686943]])

            region2 = ee.Geometry.Polygon([[75.72866443690508,31.230734958247353],[75.7286322503969,31.229927635738004],[75.72969976958483,31.22990928741898],
                                          [75.7297480493471,31.230693674877305],[75.72866443690508,31.230734958247353]]) 

            region3 = ee.Geometry.Polygon([[75.72594467696398,31.230812937897173],[75.7259124904558,31.230179924763107],[75.72738770541399,31.23015698942504],
                                          [75.72739843425005,31.230794589750076],[75.72594467696398,31.230812937897173]])

            Map.centerObject(bigger_region, 16)

            reReArgs['geometry'] = region1
            f.value = 10
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 1...')
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,' Farm 1')
                print('Adding layers for farm 1...')
                Imageurl(sdf1,region1,bigger_region,1)
                f.value = 65
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 1")

            reReArgs['geometry'] = region2
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 2...')
                andvil2,andviu2,andvin2,andvia2,sdf2 = predict(df)
                aplot(andvil2,andviu2,andvin2,andvia2,sdf2,' Farm 2')
                print('Adding layers for farm 2...')
                Imageurl(sdf2,region2,bigger_region,2)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 2")

            reReArgs['geometry'] = region3
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                print('Processing data for farm 3...')
                andvil3,andviu3,andvin3,andvia3,sdf3 = predict(df)
                aplot(andvil3,andviu3,andvin3,andvia3,sdf3,' Farm 3')
                print('Adding layers for farm 3...')
                Imageurl(sdf3,region3,bigger_region,3)
                f.value = 100
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for farm 3")
            print("Done.")
            
        elif Areas.value == 'My_Region':

            a=  Map.draw_last_feature

            if a == None:
                print('Please Draw a Polygon')
                return
            if Txtbx2.value != '':
                farm_name = Txtbx2.value
            else:
                print("Please provide a name for your farm.")
                return
            
            b = a.getInfo()['geometry']['coordinates']
            region1 = ee.Geometry.Polygon(b[0])

            bigger_region = region1.buffer(0.1)
            
            
            f.value = 10
            reReArgs['geometry'] = region1
            NDVI,EVI = ProcessImg(bigger_region,from_date,to_date,clouds_percentage)
            f.value = 50
            ndvi = getReReList(NDVI, ['Date', 'NDVI'])
            evi = getReReList(EVI, ['Date', 'EVI'])
            print('Processing data for the selected region...')
            df,st = create_df(ndvi,evi)
            if (st == "l40p"):
                andvil1,andviu1,andvin1,andvia1,sdf1 = predict(df)
                aplot(andvil1,andviu1,andvin1,andvia1,sdf1,farm_name)
                f.value = 80
                print('Adding layers...')
                Imageurl(sdf1,region1,bigger_region,0)
                if Chckbx.value==True:
                    threshold = (thrsld.value/100)
                    data = {"mail":Txtbx.value,"loc":str(b[0]),"acc_tkn":"0447379023793675","farm":Txtbx2.value,"threshold":threshold}
                    print('E-mail with Health info of the Area of Interest chosen will be sent by end of every week')
                    url = "https://ackamrxc4anoos3auilnhwzt2q.apigateway.us-ashburn-1.oci.customer-oci.com/httptest2/httptest2"
                    print('The required information is being uploaded to our database.....')
                    x = requests.post(url, json=data)
            elif (st == "40p"):
                bplot(df)
            elif (st == "100p"):
                print("No image is available for the selected region")
            f.value = 100
            print("Done.")
        
        
Submit.on_click(submit_clck)

In [35]:
def center_clck(c):
    with output:
        clear_output()
        a=  Map.draw_last_feature

        if a == None:
            print('No region has been drawn before')
            return

        b = a.getInfo()['geometry']['coordinates']
        region1 = ee.Geometry.Polygon(b[0])
        Map.centerObject(region1, 16)

Center.on_click(center_clck)

In [36]:
Landsat7_tier1_dataset = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")

In [37]:
reReArgs = {
'reducer': ee.Reducer.mean(),
'geometry': ee.Geometry.Point([0,0]),
'scale': 200}

In [38]:
output = widgets.Output()

In [39]:
Hbox1=widgets.HBox([Submit,Center])
Hbox1

In [40]:
f = widgets.IntProgress(min=0, max=100)
display(f)

IntProgress(value=0)

In [41]:
display(output)

Output()